In [ ]:
import os
import sys

def add_to_path(new_path: str):
    module_path = os.path.abspath(os.path.join(new_path))
    if module_path not in sys.path:
        sys.path.append(module_path)
    
is_colab = False

if is_colab:
    !git clone https://github.com/pdkary/Karys.git
    !cd Karys && git fetch && git pull
    !cd Karys && pip install -r requirements.txt --quiet
    add_to_path("Karys/")
    from google.colab import drive
    drive.mount("/content/drive")
else:
    !cd ../../ && pip install -r requirements.txt --quiet
    add_to_path("../../")

## Aight we doing HOT DOG / NOT HOT DOG discrimination

In [ ]:
from data.configs.ImageDataConfig import ImageDataConfig
from data.wrappers.ImageDataWrapper import ImageDataWrapper

if is_colab:
  target_path = "drive/MyDrive/Colab/Seefood/HotDog"
  noise_path = "drive/MyDrive/Colab/Seefood/NotHotDog"
else:
  target_path = "./test_input/target_images"
  noise_path = "./test_input/noise_images"

image_config = ImageDataConfig(image_shape=(64,64,3),image_type=".jpg", features_size=2, preview_rows=2, preview_cols=2)

target_data_wrapper = ImageDataWrapper.load_from_file(target_path, image_config, validation_percentage=0.1)
noise_data_wrapper = ImageDataWrapper.load_from_file(noise_path, image_config, validation_percentage=0.1)

### We only need this next one if we are loading a saved model

In [ ]:
from data.saved_models.SavedModelService import SavedModelService
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy, MSE, CategoricalCrossentropy


load_from_saved_models = True

if is_colab:
  model_reference_path = "drive/MyDrive/Colab/Seefood/models/model_ref.json"
else:
  model_reference_path = "./test_model_output/model_ref.json"

saved_model_service = SavedModelService(model_reference_path)

if load_from_saved_models:
  optimizer = Adam(learning_rate=5e-4)
  loss = CategoricalCrossentropy(from_logits=True)
  classifier = saved_model_service.load_model_by_name("SeefoodClassifier", optimizer, loss)

### ok if not loading, use this one

In [ ]:
from models.ModelWrapper import ModelWrapper
from tensorflow.keras.layers import Conv2D, Dense, LeakyReLU, MaxPooling2D, Flatten, Activation, BatchNormalization
from tensorflow_addons.layers import InstanceNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy, MSE, CategoricalCrossentropy

if not load_from_saved_models:
  a = 0.08
  disc_layers = [
      Conv2D(64,3), LeakyReLU(a),
      Conv2D(64,3), BatchNormalization(), LeakyReLU(a),
      MaxPooling2D(),
      Conv2D(128,3), LeakyReLU(a),
      Conv2D(128,3), BatchNormalization(), LeakyReLU(a),
      MaxPooling2D(),
      Conv2D(256,3), LeakyReLU(a),
      Conv2D(256,3), LeakyReLU(a),
      Conv2D(256,3), BatchNormalization(), LeakyReLU(a),
      MaxPooling2D(),
      Conv2D(512,3, padding="same"), LeakyReLU(a),
      Conv2D(512,3, padding="same"), LeakyReLU(a),
      Conv2D(512,3, padding="same"), BatchNormalization(), LeakyReLU(a),
      MaxPooling2D(),
      Conv2D(512,3, padding="same"), LeakyReLU(a),
      Conv2D(512,3, padding="same"), LeakyReLU(a),
      Conv2D(512,3, padding="same"), BatchNormalization(), LeakyReLU(a),
      # MaxPooling2D(),
      Flatten(),
      Dense(4096), Activation('relu'),
      Dense(1000), Activation('relu'),
      Dense(image_config.features_size), Activation('softmax'),
  ]
  optimizer = Adam(learning_rate=5e-4)
  loss = CategoricalCrossentropy(from_logits=True)

  classifier = ModelWrapper(image_config.image_shape, [image_config.features_size], disc_layers, optimizer, loss, flatten_input=False)
  classifier.build()

In [ ]:
from plotting.TrainPlotter import TrainPlotter
from trainers.ClassificationTrainer import ClassificationTrainer

epochs=100
trains_per_test=50
batch_size = 25
batches_per_loop = 1

train_columns = ["Hot Dog Train Loss","Not Hot Dog Train Loss", "Hot Dog Test Loss", "Not Hot Dog Test Loss"]
loss_plot = TrainPlotter(moving_average_size=100,labels=train_columns)
trainer = ClassificationTrainer(classifier, target_data_wrapper, noise_data_wrapper)

if is_colab:
  image_output_path = "drive/MyDrive/Colab/Seefood/output"
else:
  image_output_path = "./test_output"

t_test_loss = 0
n_test_loss = 0
for i in range(epochs):
  loss_plot.start_epoch()  
  t_loss, n_loss = trainer.train(batch_size, batches_per_loop)

  if i % trains_per_test == 0 and i != 0:
    t_test_loss, n_test_loss = trainer.test(4,1)
    test_output_filename = image_output_path + "/train-" + str(i) + ".jpg"
    target_data_wrapper.save_classified_images(test_output_filename, trainer.most_recent_target_output, trainer.most_recent_noise_output)
      
  loss_plot.batch_update([t_loss,n_loss,t_test_loss,n_test_loss])
  loss_plot.log_epoch()

In [ ]:
if is_colab:
  model_output_path = "drive/MyDrive/Colab/Seefood/models"
else:
  model_output_path = "./test_model_output"

saved_model_service.save_model("SeefoodClassifier",model_output_path, classifier)